### Sesión 13: Árboles de decisión (I)

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

<div id="About_dataset">
    <h2>Sobre el dataset</h2>
    Imagina que eres un investigador médico que recopila datos para un estudio. Has recopilado datos sobre un conjunto de pacientes, los cuales padecían la misma enfermedad. Durante el curso de su tratamiento, cada paciente respondió a uno de los 5 medicamentos, Drug A, Drug B, Drug c, Drug x and y. 
    <br>
    <br>
    Parte de tu trabajo es construir un modelo para descubrir qué medicamento podría ser apropiado para un futuro paciente con la misma enfermedad. Los conjuntos de características de este conjunto de datos son la edad (Age), el sexo (Sex), la presión arterial (BP) y el colesterol (Cholesterol) de los pacientes, y el <i>target</i> es el medicamento al que respondió cada paciente.
    <br>
    <br>
    Esta es una muestra de clasificador binario, y puedes usar la parte de entrenamiento del conjunto de datos para construir un árbol de decisión y luego usarla para predecir la clase de un paciente desconocido, o para prescribirla a un nuevo paciente.
</div>


Ahora, lee los datos usando *dataframe* de pandas:

In [ ]:
my_data = pd.read_csv("data/drug200.csv", delimiter=",")
my_data.head()

In [ ]:
my_data.shape

<div href="pre-processing">
    <h2>Pre-procesamiento</h2>
</div>

Usando <b>my_data</b> como los datos de Drug.csv leídos por pandas, declara las siguientes variables: <br>

<ul>
    <li> <b> X </b> como la <b> Matriz de características </b> (data de my_data) </li>
    <li> <b> y </b> como el <b> vector de respuesta (target) </b> </li>
</ul>

Elimina la columna que contiene el nombre del *target*, ya que no contiene valores numéricos.

In [ ]:
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']]
X[0:5]

Como puedes observar, algunas características de este conjunto de datos son categóricas, como __Sex__ o __BP__. Desafortunadamente, los árboles de decisión de Sklearn no manejan variables categóricas.

Convierte las varibles categóricas en variables ficticias(dummy)/indicadoras.

In [ ]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X.values[:,1] = le_sex.transform(X.values[:,1]) 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X.values[:,2] = le_BP.transform(X.values[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X.values[:,3] = le_Chol.transform(X.values[:,3]) 

X[0:5]


In [ ]:
X.shape

In [ ]:
X=pd.get_dummies(X, columns=['Sex', 'BP', 'Cholesterol'])
X

Ahora podemos llenar la variable objetivo (<i>target</i>):

In [ ]:
y = my_data["Drug"]
y[0:5]

<hr>

<div id="setting_up_tree">
    <h2>Configurando el Árbol de decisión</h2>
    Nosotros estaremos usando <b> la división de train/test</b> en nuestro <b>árbol de decisión</b>. Vamos a importar <b>train_test_split</b> de <b>sklearn.cross_validation</b>.
</div>

In [ ]:
from sklearn.model_selection import train_test_split

Ahora <b> train_test_split </b> devolverá 4 parámetros diferentes. Los nombraremos:<br>
X_trainset, X_testset, y_trainset, y_testset <br> <br>
El <b> train_test_split </b> necesitará los parámetros: <br>
X, y, test_size=0.3, and random_state=3. <br> <br>
La <b>X</b> y <b>y</b> son las matrices requeridas antes de la división, el <b>test_size</b> representa la proporción del conjunto de datos de prueba, y el <b>random_state</b> asegura que obtengamos las mismas divisiones.

In [ ]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

In [ ]:
print(X_trainset.shape)
print(X_testset.shape)
print(y_trainset.shape)
print(y_testset.shape)

<hr>

<div id="modeling">
    <h2>Modelando</h2>
    Primero crearemos una instancia de <b>DecisionTreeClassifier</b> llamada <b>drugTree</b>.<br>
    Dentro del clasificador, especifica <i> criterion="entropy" </i> para que podamos ver la ganancia de información de cada nodo.
</div>

In [ ]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree # it shows the default parameters

A continuación, ajustaremos los datos con la matriz de características de entrenamiento <b> X_trainset </b> y el vector de respuesta de entrenamiento <b> y_trainset </b>:

In [ ]:
drugTree.fit(X_trainset,y_trainset)

In [ ]:
#Importancia de variables

drugTree.feature_importances_

In [ ]:
#Una mejor forma de ver la importancia de variables

pd.Series(data=drugTree.feature_importances_, index=X.columns)

<hr>

<div id="prediction">
    <h2>Predicción</h2>
    Vamos a hacer algunas <b>predicciones</b> en el conjunto de datos de prueba y a almacenarlas en una variable llamada <b>predTree</b>.
</div>

In [ ]:
predTree = drugTree.predict(X_testset)

Puedes imprimir <b>predTree</b> y <b>y_testset</b> si deseas comparar visualmente la predicción con los valores reales:

In [ ]:
print (predTree [0:5])
print (y_testset [0:5])

## Graficamos el arbol obtenido

In [ ]:
!pip install graphviz

In [ ]:
#Gráfico del árbol de decisión

from graphviz import Source
from sklearn.tree import export_graphviz
from IPython.display import Image
from pydotplus import graph_from_dot_data

In [ ]:
dot_data = export_graphviz(drugTree, filled = True,
                          special_characters = True)

graph = graph_from_dot_data(dot_data)
graph.write_png('tree0.png')
Image(graph.create_png())

<hr>

<div id="evaluation">
    <h2>Evaluación</h2>
    A continuación, vamos a importar <b>metrics</b> de sklearn y verificar la precisión (<i>Accuracy</i>) de nuestro modelo:
</div>

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

__Puntaje de la precisión de la clasificación__ calcula la precisión del subconjunto: el conjunto de etiquetas predicho para una muestra debe coincidir exactamente con el conjunto de etiquetas correspondiente en y_true.

En la clasificación de múltiples etiquetas, la función devuelve la precisión del subconjunto. Si el conjunto completo de etiquetas pronosticadas para una muestra coincide estrictamente con el conjunto real de etiquetas, la precisión del subconjunto es 1.0; de lo contrario es 0.0.

In [ ]:
for crit in ['entropy','gini']:
    for mDepth in [2,4,6,8,10]:
        drugTree_loop = DecisionTreeClassifier(criterion=crit, max_depth = mDepth)
        drugTree_loop.fit(X_trainset,y_trainset)
        drugTree_loop = drugTree_loop.predict(X_testset)
        print("DecisionTrees's Accuracy: "+crit+" "+str(mDepth)+" "+str(metrics.accuracy_score(y_testset, drugTree_loop)))
        